# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
# NOTE = run in terminal to display gmaps (before activating jupyter notebook):  jupyter nbextension enable --py gmaps

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
#Load the csv exported in Part I to a DataFrame
weather_data_df = pd.read_csv("output_data_CCB/cities.csv").drop('Unnamed: 0', axis =1)
weather_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,‘Āhuimanu,21.44,-157.84,87.80,66,20,12.75,US,1602466119
1,Hermanus,-34.42,19.23,53.01,78,57,4.00,ZA,1602465941
2,Port Alfred,-33.59,26.89,53.78,68,5,7.20,ZA,1602466123
3,Jamestown,42.10,-79.24,62.01,72,90,6.93,US,1602465971
4,Saskylakh,71.92,114.08,17.51,95,84,8.14,RU,1602466127
...,...,...,...,...,...,...,...,...,...
531,Talnakh,69.49,88.40,24.80,92,40,22.37,RU,1602467237
532,Roald,62.58,6.13,44.60,93,83,3.36,NO,1602467239
533,Huaidian,33.43,115.03,66.04,72,97,7.87,CN,1602467241
534,Claresholm,50.03,-113.59,46.99,45,6,8.05,CA,1602467243


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.

gmaps.configure(api_key=g_key)

locations = weather_data_df[["Lat", "Lng"]].astype(float)

humidity = weather_data_df["Humidity"].astype(float)

In [9]:
# Add Heatmap layer to map

fig = gmaps.figure(layout={
        'width': '900px',
        'height': '600px',
        'border': '1px solid black'})

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Create new dataframe by narrowing down the cities to fit ideal weather conditions

hotel_df = weather_data_df.loc[(weather_data_df['Max Temp'].between(78,85)) &
                                     (weather_data_df['Cloudiness'] < 10) &
                                      (weather_data_df['Humidity'] < 40) &
                                    (weather_data_df['Wind Speed'] < 10)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
130,Laguna,38.42,-121.42,80.60,14,1,9.17,US,1602466240
189,Biltine,14.53,20.92,83.55,23,0,4.34,TD,1602466504
361,Zinder,13.81,8.99,83.62,18,0,4.38,NE,1602466888
427,Siwāna,25.63,72.42,81.27,31,0,0.27,IN,1602467015
491,Maiduguri,11.85,13.16,79.63,24,0,4.99,NG,1602467151
524,Wagar,16.15,36.20,79.47,36,0,7.67,SD,1602467223
525,Pahrump,36.21,-115.98,82.40,14,1,6.93,US,1602467225


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#Add a "Hotel Name" column to the DataFrame

hotel_df['Hotel Name'] = ""
hotel_df


C:\Users\cch31\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
130,Laguna,38.42,-121.42,80.60,14,1,9.17,US,1602466240,
189,Biltine,14.53,20.92,83.55,23,0,4.34,TD,1602466504,
361,Zinder,13.81,8.99,83.62,18,0,4.38,NE,1602466888,
427,Siwāna,25.63,72.42,81.27,31,0,0.27,IN,1602467015,
491,Maiduguri,11.85,13.16,79.63,24,0,4.99,NG,1602467151,
524,Wagar,16.15,36.20,79.47,36,0,7.67,SD,1602467223,
525,Pahrump,36.21,-115.98,82.40,14,1,6.93,US,1602467225,


In [12]:
#Set parameters to search for hotels with 5000 meters
params = {
    'radius' : 5000,
    'types' : 'lodging', 
    'keyword' : 'hotel',
    'key': g_key
}

# Use the lat/lng we recovered to identify ideal locations
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Hit the Google Places API for each city's coordinates
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_nearby = requests.get(base_url, params=params)
    
    # convert to json
    hotel_nearby = hotel_nearby.json()
    #print(json.dumps(hotel_nearby, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_nearby["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")


C:\Users\cch31\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.


In [13]:
#display new dataframe with hotel name
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
130,Laguna,38.42,-121.42,80.60,14,1,9.17,US,1602466240,Hilton Garden Inn Sacramento Elk Grove
189,Biltine,14.53,20.92,83.55,23,0,4.34,TD,1602466504,Hôtel Trois Étoiles de Biltine
361,Zinder,13.81,8.99,83.62,18,0,4.38,NE,1602466888,Hôtel Royal
427,Siwāna,25.63,72.42,81.27,31,0,0.27,IN,1602467015,Kirti Hotel
491,Maiduguri,11.85,13.16,79.63,24,0,4.99,NG,1602467151,Satus Hotel
524,Wagar,16.15,36.20,79.47,36,0,7.67,SD,1602467223,
525,Pahrump,36.21,-115.98,82.40,14,1,6.93,US,1602467225,Holiday Inn Express & Suites Pahrump


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', width='900px'))